In [335]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import ast

In [336]:
df = pd.read_csv('movies_metadata.csv')
df.head()

/var/folders/6t/6mjcspy10fzf5dhd8gb75g280000gn/T/ipykernel_66020/3991525872.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [337]:
columns = df.columns.tolist()
print("Columns in the dataset:")
print(columns)

Columns in the dataset:
['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']


The only columns we care about are:
- adult: categorical
- genres: categorical
- original_language: categorical
- budget: numerical
- popularity: numerical
- revenue: numerical
- runtime: numerical
- vote_average: numerical

In [338]:
columns_to_keep = ['adult', 'genres', 'original_language', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average']

df = df[columns_to_keep]
print("\nDataFrame with selected columns:")
df.head()


DataFrame with selected columns:


,adult,genres,original_language,budget,popularity,revenue,runtime,vote_average
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,30000000,21.946943,373554033.0,81.0,7.7
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,65000000,17.015539,262797249.0,104.0,6.9
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,0,11.7129,0.0,101.0,6.5
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,16000000,3.859495,81452156.0,127.0,6.1
4,False,"[{'id': 35, 'name': 'Comedy'}]",en,0,8.387519,76578911.0,106.0,5.7


In [339]:
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')

In [340]:
# Filter out movies where 'budget' is 0
df = df[(df['budget'] != 0) & (df['revenue'] != 0) & (df['runtime'] != 0)]
df.reset_index(drop=True, inplace=True)
df.head(20)

,adult,genres,original_language,budget,popularity,revenue,runtime,vote_average
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,30000000.0,21.946943,373554033.0,81.0,7.7
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,65000000.0,17.015539,262797249.0,104.0,6.9
2,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,16000000.0,3.859495,81452156.0,127.0,6.1
3,False,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",en,60000000.0,17.924927,187436818.0,170.0,7.7
4,False,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",en,35000000.0,5.23158,64350171.0,106.0,5.5
5,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",en,58000000.0,14.686036,352194034.0,130.0,6.6
6,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,62000000.0,6.318445,107879496.0,106.0,6.5
7,False,"[{'id': 36, 'name': 'History'}, {'id': 18, 'na...",en,44000000.0,5.092,13681765.0,192.0,7.1
8,False,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",en,98000000.0,7.284477,10017322.0,119.0,5.7
9,False,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",en,52000000.0,10.137389,116112375.0,178.0,7.8


In [341]:
df.shape

(5373, 8)

In [342]:
# filter out movies whose genres is an empty list
df = df[df['genres'].apply(lambda x: isinstance(x, str) and len(ast.literal_eval(x)) > 0)]
df.reset_index(drop=True, inplace=True)
df.shape

(5362, 8)

In [343]:
y = df['revenue'] # we would like to predict revenue, separate it from the df

In [344]:
# Create a copy of the df for encoding
encoded_df = df.copy()

In [345]:
# one hot encode the 'adult' column
encoded_df['adult'] = encoded_df['adult'].apply(lambda x: 1 if x == 'True' else 0)
encoded_df.head()

,adult,genres,original_language,budget,popularity,revenue,runtime,vote_average
0,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,30000000.0,21.946943,373554033.0,81.0,7.7
1,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,65000000.0,17.015539,262797249.0,104.0,6.9
2,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,16000000.0,3.859495,81452156.0,127.0,6.1
3,0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",en,60000000.0,17.924927,187436818.0,170.0,7.7
4,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",en,35000000.0,5.23158,64350171.0,106.0,5.5


In [346]:
unique_languages = encoded_df['original_language'].unique()
unique_languages

array(['en', 'fr', 'cn', 'ru', 'es', 'zh', 'sv', 'it', 'de', 'ja', 'fa',
       'pt', 'xx', 'da', 'ko', 'el', 'hi', 'pl', 'af', 'fi', 'th', 'nl',
       'ro', 'vi', 'bm', 'he', 'no', 'ta', 'sr', 'nb', 'ca', 'id',
       '104.0', 'tr', 'ml', 'hu', 'kn', 'is', '68.0', 'te', 'mr', '82.0',
       'ur'], dtype=object)

In [347]:
# get rid of the rows where original_language is '68.0' and '82.0'
encoded_df = encoded_df[~encoded_df['original_language'].isin(['68.0', '82.0', '104.0'])]

In [348]:
unique_languages = encoded_df['original_language'].unique()
unique_languages

array(['en', 'fr', 'cn', 'ru', 'es', 'zh', 'sv', 'it', 'de', 'ja', 'fa',
       'pt', 'xx', 'da', 'ko', 'el', 'hi', 'pl', 'af', 'fi', 'th', 'nl',
       'ro', 'vi', 'bm', 'he', 'no', 'ta', 'sr', 'nb', 'ca', 'id', 'tr',
       'ml', 'hu', 'kn', 'is', 'te', 'mr', 'ur'], dtype=object)

In [349]:
top_languages = encoded_df['original_language'].value_counts().head(5).index
top_languages

Index(['en', 'hi', 'fr', 'ru', 'ja'], dtype='object', name='original_language')

The top 5 languages are English, Hindi, French, Russian, and Japanese.

In [350]:
encoded_df['original_language'] = encoded_df['original_language'].apply(
    lambda x: x if x in top_languages else 'other'
)
# find the number of rows whose original_language is 'Other'
num_other_languages = (encoded_df['original_language'] == 'other').sum()
num_other_languages

np.int64(281)

There are 282 movies whose language is not the top 5 language.

In [351]:
# one hot encode original_language column
encoded_df = pd.get_dummies(encoded_df, columns=['original_language'], dtype = int)
encoded_df.head()

,adult,genres,budget,popularity,revenue,runtime,vote_average,original_language_en,original_language_fr,original_language_hi,original_language_ja,original_language_other,original_language_ru
0,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000.0,21.946943,373554033.0,81.0,7.7,1,0,0,0,0,0
1,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000.0,17.015539,262797249.0,104.0,6.9,1,0,0,0,0,0
2,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000.0,3.859495,81452156.0,127.0,6.1,1,0,0,0,0,0
3,0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",60000000.0,17.924927,187436818.0,170.0,7.7,1,0,0,0,0,0
4,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",35000000.0,5.23158,64350171.0,106.0,5.5,1,0,0,0,0,0


In [352]:
# Convert the string representation of genres to a list of dictionaries
encoded_df['genres'] = encoded_df['genres'].apply(ast.literal_eval)

# get all the unique genres
# Extract just the genre names for each movie
genre_lists = encoded_df['genres'].apply(lambda x: [genre['name'] for genre in x])
genre_lists.head()

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2            [Comedy, Drama, Romance]
3    [Action, Crime, Drama, Thriller]
4       [Action, Adventure, Thriller]
Name: genres, dtype: object

In [353]:
# Create a MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform the genre lists into a binary matrix
genre_matrix = mlb.fit_transform(genre_lists)

# Create a DataFrame with the binary columns
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_)

# Concatenate with the original dataframe
encoded_df = pd.concat([encoded_df.drop('genres', axis=1), genre_df], axis=1)

encoded_df.head()

,adult,budget,popularity,revenue,runtime,vote_average,original_language_en,original_language_fr,original_language_hi,original_language_ja,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,0.0,30000000.0,21.946943,373554033.0,81.0,7.7,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,65000000.0,17.015539,262797249.0,104.0,6.9,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,16000000.0,3.859495,81452156.0,127.0,6.1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,60000000.0,17.924927,187436818.0,170.0,7.7,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,35000000.0,5.23158,64350171.0,106.0,5.5,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [354]:
# filter out movies that has NaN values in any of the columns
encoded_df = encoded_df.dropna()
encoded_df.reset_index(drop=True, inplace=True)
encoded_df.shape

(5355, 32)

In [355]:
encoded_df.tail()

,adult,budget,popularity,revenue,runtime,vote_average,original_language_en,original_language_fr,original_language_hi,original_language_ja,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
5350,0.0,2196531.0,0.947509,2411594.0,107.0,5.6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5351,0.0,60000000.0,50.903593,71000000.0,95.0,5.7,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5352,0.0,50000000.0,33.694599,66913939.0,86.0,5.8,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5353,0.0,11000000.0,40.796775,184770205.0,111.0,7.4,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5354,0.0,12000000.0,1.323587,19000000.0,185.0,6.9,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [356]:
columns = encoded_df.columns.tolist()
print("\nColumns in the encoded DataFrame:")
print(columns)
print('Number of columns in the encoded DataFrame:', len(columns))


Columns in the encoded DataFrame:
['adult', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'original_language_en', 'original_language_fr', 'original_language_hi', 'original_language_ja', 'original_language_other', 'original_language_ru', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']
Number of columns in the encoded DataFrame: 32


In [357]:
scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(encoded_df).round(2)
scaled_X

array([[0.  , 0.08, 0.04, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.17, 0.03, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.04, 0.01, ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.13, 0.06, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.03, 0.07, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.03, 0.  , ..., 0.  , 0.  , 0.  ]])

In [358]:
Phi = np.hstack((scaled_X, np.ones((scaled_X.shape[0], 1))))
Phi

array([[0.  , 0.08, 0.04, ..., 0.  , 0.  , 1.  ],
       [0.  , 0.17, 0.03, ..., 0.  , 0.  , 1.  ],
       [0.  , 0.04, 0.01, ..., 0.  , 0.  , 1.  ],
       ...,
       [0.  , 0.13, 0.06, ..., 0.  , 0.  , 1.  ],
       [0.  , 0.03, 0.07, ..., 0.  , 0.  , 1.  ],
       [0.  , 0.03, 0.  , ..., 0.  , 0.  , 1.  ]])